In [1]:
"""
Feature: temperature
Extract max temperature from chartevents (ICU) and vitalsign (ED)
"""
import pandas as pd
import os
from config_paths import *
from utils import drop_columns

In [2]:
# Read dataset
df = pd.read_csv(dataset_path)
initial_row_count = len(df)

# Load ICU chartevents in chunks
temp_itemids = [223761, 223762]  # Temperature item IDs
temp_max_list = []

try:
    chunk_iter = pd.read_csv(
        os.path.join(icu_path, "chartevents.csv"),
        usecols=["subject_id", "hadm_id", "itemid", "valuenum"],
        chunksize=1000000,
        low_memory=False
    )
    
    for chunk in chunk_iter:
        temp = chunk[chunk["itemid"].isin(temp_itemids)].copy()
        temp = temp.dropna(subset=["valuenum"])
        if temp.empty:
            continue
        
        # Convert Celsius to Fahrenheit if needed
        temp["temp_f"] = temp["valuenum"]
        mask_c = temp["itemid"] == 223762
        temp.loc[mask_c, "temp_f"] = temp.loc[mask_c, "valuenum"] * 9/5 + 32
        
        # Fix decimal shift errors
        temp.loc[(temp["temp_f"] > 110) & (temp["temp_f"] <= 300), "temp_f"] /= 10
        temp.loc[(temp["temp_f"] > 300) & (temp["temp_f"] <= 1000), "temp_f"] /= 100
        
        # Filter reasonable values
        temp = temp[(temp["temp_f"] >= 80) & (temp["temp_f"] <= 110)]
        
        if not temp.empty:
            temp_max_chunk = temp.groupby("hadm_id")["temp_f"].max()
            temp_max_list.append(temp_max_chunk)
    
    if temp_max_list:
        temp_max_icu = (
            pd.concat(temp_max_list)
            .groupby("hadm_id")
            .max()
            .reset_index()
            .rename(columns={"temp_f": "temperature_icu"})
        )
    else:
        temp_max_icu = pd.DataFrame(columns=["hadm_id", "temperature_icu"])
except:
    temp_max_icu = pd.DataFrame(columns=["hadm_id", "temperature_icu"])

# Load ED vitalsign
try:
    vitalsign = pd.read_csv(os.path.join(ed_path, "vitalsign.csv"))
    ed_stays = pd.read_csv(os.path.join(ed_path, "edstays.csv"))
    
    ed_vitals_temp = ed_stays.merge(
        vitalsign[["subject_id", "stay_id", "temperature"]],
        on=["subject_id", "stay_id"],
        how="left"
    )
    
    temp_max_ed = (
        ed_vitals_temp.groupby(["subject_id", "hadm_id"])["temperature"]
        .max()
        .reset_index()
        .rename(columns={"temperature": "temperature_ed"})
    )
except:
    temp_max_ed = pd.DataFrame(columns=["subject_id", "hadm_id", "temperature_ed"])

# Merge
df = df.merge(temp_max_icu, on="hadm_id", how="left")
if not temp_max_ed.empty:
    df = df.merge(temp_max_ed, on=["subject_id", "hadm_id"], how="left")

# Combine: prefer ICU, then ED
df["temperature"] = df["temperature_icu"].fillna(df.get("temperature_ed", pd.Series()))

# Drop intermediate columns
df.drop(columns=["temperature_icu", "temperature_ed"], errors="ignore", inplace=True)

# Save
df.to_csv(dataset_path, index=False)

# Diagnostic output
print(f"\nTotal rows: {len(df)}")
print(f"Expected total: {initial_row_count}")
if len(df) == initial_row_count:
    print(f"✓ Row count matches expected total ({initial_row_count})")
else:
    print(f"⚠ Row count mismatch! Expected {initial_row_count}, got {len(df)} (difference: {len(df) - initial_row_count})")
print(f"Dataset shape: {df.shape}")
print(f"Feature 'temperature' added. Non-null count: {df['temperature'].notna().sum()}")
print(f"Dataset shape: {df.shape}")




Total rows: 158020
Expected total: 158020
✓ Row count matches expected total (158020)
Dataset shape: (158020, 93)
Feature 'temperature' added. Non-null count: 110094
Dataset shape: (158020, 93)


In [3]:
df.columns

Index(['subject_id', 'hadm_id', 'cauti_type', 'cauti_icd_codes',
       'other_uti_icd_codes', 'remaining_icd_codes', 'gender', 'anchor_age',
       'admittime', 'dischtime', 'admission_type', 'admission_location',
       'discharge_location', 'race', 'catheter_procedures',
       'catheter_procedures_ir', 'catheter_used_in_procedures_ir',
       'catheter_insertion_date_ir', 'catheter_removal_date_ir',
       'catheter_outputevents', 'catheter_used_in_output_events',
       'datetimeevents', 'catheter_used_in_datetime_events',
       'insertion_date_from_datetimeevents',
       'removal_date_from_datetimeevents', 'catheter_procedure_events',
       'catheter_used_in_procedure_events', 'catheter_size_from_chartevents',
       'index', 'catheter_clinical_notes', 'catheter_used_in_clinical_notes',
       'catheter_present', 'final_cauti_flag', 'final_insertion_date',
       'final_removal_date', 'catheter_duration_days', 'BMI_in_admission',
       'BMI_computed', 'BMI_last_year', 'BMI', 

In [4]:
# cols_to_drop = [
#  "temperature"
# ]
# df = drop_columns(df, cols_to_drop)
# df.to_csv(dataset_path, index=False)